In [ ]:
from typing import Callable
import time

def my_decorator(func: Callable):
    def wrapper():
        print(f"Something was done before {func.__name__} was executed.")
        func()
        print(f"Something was done after {func.__name__} was executed.")
    return wrapper

def timer(func: Callable):
    def wrapper():
        time_0 = time.time()
        func()
        time_1 = time.time()
        print(f"{func.__name__} took {(time_1 - time_0):.2f} seconds to execute.")
    return wrapper

@my_decorator
def hello_world_1():
    print("hello world!")

@timer
def hello_world_2():
    tmp = 0
    # waste time
    for i in range(114514810):
        tmp += i
    print("hello world!")

if __name__ == '__main__':
    hello_world_1()
    hello_world_2()

In [8]:
# decorator of func with any args
from typing import Optional, Callable


def deco_args(func: Callable):
    def wrapper(*args, **kwargs):
        print("Positional Args:")
        for arg in args:
            print(f"    {arg}")
        print("Keyword Args:")
        for k, v in kwargs.items():
            print(f"    {k}={v}")
        ret = func()
        print(f"Return Value: {ret}")
    return wrapper


@deco_args
def hello_world_3(name: str = "world", *, msg: Optional[str] = None):
    print(f"Hello, {name}!")
    if msg is not None:
        print(msg)
    return "Over!"


hello_world_3("YAJU", msg="&U")

Positional Args:
    YAJU
Keyword Args:
    msg=&U
Hello, world!
Return Value: Over!


In [10]:
# metadata problem
from functools import wraps

def deco_with_wraps(func): 
    @wraps(func)
    def wrapper(*args, **kwargs): 
        result = func(*args, **kwargs) 
        return result 
    return wrapper 
    
def deco_without_wraps(func): 
    def wrapper(*args, **kwargs): 
        result = func(*args, **kwargs) 
        return result 
    return wrapper 

@deco_with_wraps
def test_1(a, b): 
    return a + b 
    
@deco_without_wraps
def test_2(a, b): 
    return a + b 

print(test_1.__name__)
print(test_2.__name__)

test_1
wrapper


In [11]:
# metadata problem
from functools import wraps

def deco_with_wraps(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result
    return wrapper

@deco_with_wraps
def test(a, b):
    return a + b

print(test.__name__)

test


In [13]:
# 编写一个装饰器，将函数返回的字符串全部转为大写
def to_upper(func: Callable[..., str]):
    def wrapper(*args, **kwargs):
        ret = func().upper()
        return ret
    return wrapper

@to_upper
def hello_world():
    return 'hello, world!'
    
hello_world()  



'HELLO, WORLD!'

In [16]:
# 创建一个装饰器，在函数执行前验证所有数字参数必须大于0
def validate_positive(func: Callable):
    def wrapper(*args, **kwargs):
        for arg in args:
            if isinstance(arg, (int, float)) and arg <= 0:
                raise ValueError(f"参数 {arg} 不大于0.")
        for k, v in kwargs.items():
            if isinstance(v, (int, float)) and v <= 0:
                raise ValueError(f"参数 {k}={v} 不大于0.")
        return func(*args, **kwargs)
        
    return wrapper

@validate_positive
def test(a, b, *, d):
    print(a, b, d)

test(1, "haha", d=-1)

ValueError: 参数 d=-1 不大于0.

In [ ]:
# 实现一个装饰器，当函数抛出指定异常时自动重试最多3次
from time import sleep
from random import randint
from typing import Callable

def retry(exception: Exception, max_attempts: int = 3):
    def decorator(func: Callable):
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < max_attempts:
                try:
                    # simulate processing time
                    sleep(1)
                    return func()
                except exception as e:
                    attempts += 1
                    print(f"{type(e).__name__}: {e}")
                    print(f"{func.__name__} 执行失败。正在重试…… 第 {attempts} 次")
            raise ValueError(f"{func.__name__}执行失败。达到最大尝试次数。")
            
        return wrapper
    return decorator
    
    
@retry(ValueError, max_attempts=5)
def test():
    if (ret := randint(0, 9)) < 5:
        raise ValueError("看来您运气不佳。")
    else:
        return ret
    
test()
    

ValueError: 看来您运气不佳。
test 执行失败。正在重试…… 第 1 次
ValueError: 看来您运气不佳。
test 执行失败。正在重试…… 第 2 次


6

In [23]:
# 编写一个装饰器，可以指定前缀和后缀来包装函数输出
def add_prefix_suffix(prefix: str = "", suffix: str = ""):
    def decorator(func: Callable):
        def wrapper(*args, **kwargs):            
            ret = func()
            return f"{prefix}{ret}{suffix}"
        return wrapper
    return decorator

@add_prefix_suffix(prefix="PREFIX: ", suffix="  - suffix")
def test():
    return "hello, world"

print(test())


PREFIX: hello, world  - suffix


In [1]:
# 创建一个缓存装饰器，可以设置缓存过期时间(秒)
# 那是得用惰性删除策略了
from typing import Union, Callable
from datetime import datetime, timedelta
from time import sleep
from functools import wraps
import inspect

# def cached(ttl: Union[int, float] = 30):
#     def decorator(func: Callable):
#         cache = {}
#         def wrapper(*args, **kwargs):
#             # 但是这样对复杂对象参数有点不太友好。hash一下？但是能保证参数是Hashable吗？
#             args_comb = "&".join(
#                 [str(arg) for arg in args] + [f"{k}={v}" for k, v in kwargs.items()])
#             if cache.get(args_comb, None) is not None and datetime.now() < cache[args_comb]['expired']:
#                 print("读取缓存值")
#             else:
#                 print("更新缓存值")
#                 cache[args_comb] = {
#                     "value": func(*args, **kwargs),
#                     "expired": datetime.now() + timedelta(seconds=ttl)
#                 }
#             return cache[args_comb]['value']
                
#         return wrapper
#     return decorator

def cached(ttl: Union[int, float] = 30):
    def decorator(func: Callable):
        cache = {}
        @wraps(func)
        def wrapper(*args, **kwargs):
            sig = inspect.signature(func)
            bound_args = sig.bind(*args, **kwargs)
            bound_args.apply_defaults()
            
            args_key = (bound_args.args, frozenset(bound_args.kwargs.items()))
            
            if cache.get(args_key, None) is not None and datetime.now() < cache[args_key]['expired']:
                print("读取缓存值")
            else:
                print("更新缓存值")
                cache[args_key] = {
                    "value": func(*args, **kwargs),
                    "expired": datetime.now() + timedelta(seconds=ttl)
                }
            return cache[args_key]['value']
                
        return wrapper
    return decorator

@cached(ttl=3)
def test(a, b):
    return a * b


test(2, 3)
test(2, 3)
sleep(3)
test(2, 3)

更新缓存值
读取缓存值
更新缓存值


6

In [27]:
from datetime import datetime, timedelta

cur = datetime.now()
print(cur)
print(cur + timedelta(seconds=30))
print(cur < (cur + timedelta(seconds=30)))

2025-04-11 02:50:57.331164
2025-04-11 02:51:27.331164
True


In [ ]:
# 实现一个装饰器，可以限制函数在特定时间段内才能执行
from datetime import datetime

    
def limit_time_range(start: datetime, end: datetime):
    def decorator(func: Callable):
        def wrapper(*args, **kwargs):
            if start < datetime.now() < end:
                return func(*args, **kwargs)
            else:
                format_str = "%Y/%m/%d %H:%M:%S"
                raise RuntimeError(f"不在函数可执行时间（{start.strftime(format_str)} ~ {end.strftime(format_str)}）内")
        return wrapper
    return decorator

@limit_time_range(datetime.now() - timedelta(seconds=30), datetime.now())
# @limit_time_range(datetime.now(), datetime.now() + timedelta(seconds=30))
def test():
    return "hello, world"

print(test())


    

RuntimeError: 不在函数可执行时间（2025/04/11 03:10:23 ~ 2025/04/11 03:10:53）内

: 

In [4]:
def double_result(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs) * 2
    return wrapper

def add_one(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs) + 1
    return wrapper

@double_result
@add_one
def calculate(x):
    return x * 3

# 期望: (x*3 + 1) * 2
print(calculate(5))  # 当前输出是什么？如何修正？

32
